### Import Statements

In [ ]:
from sys import path
from os.path import dirname, abspath
path.append(dirname(dirname(dirname(abspath("__file__")))))

In [ ]:
# for NER model
from transformers import AutoModel
from torchcrf import CRF
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from scripts.utils_bert import *
from scripts.metrics import f1score
import pytorch_lightning as pl
from multiprocessing import cpu_count
from platform import system
from os import environ
from pickle import load

# for classification model
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

# set the environment to run tokenizers in parallel
environ["TOKENIZERS_PARALLELISM"] = "false"
pl.seed_everything(seed=42)

### NER Model (Definition)
#### NER model is only used in inference mode by loading the saved_weights
#### Hence, all training elements of the model can be removed

In [ ]:
BATCH_SIZE = 32
N_JOBS = cpu_count() if system() != "Windows" else 0

BERT_TYPE = "roberta-base"
TAG2IDX = {'B': 0, 'I': 1, 'O': 2, 'E': 3, 'S': 4, '<': 5, ">":6, "$": 7}

In [ ]:
class BERT_NER(pl.LightningModule):
    def __init__(self, 
                 bert_type=BERT_TYPE, 
                 num_tags=len(TAG2IDX),
                 test_dataset=None):
        
        super().__init__()
        self.bert = AutoModel.from_pretrained(bert_type)
        self.crf = CRF(num_tags=num_tags, batch_first=True)
        self.fc = nn.Linear(768, num_tags)
        ## Hyperparameters ##
        self.batch_size = BATCH_SIZE
        ## Datasets ##
        self.test_dataset = test_dataset


    def test_dataloader(self):
        return DataLoader(self.test_dataset, 
                          batch_size=self.batch_size,
                          num_workers=N_JOBS,
                          shuffle=False,
                          drop_last=False)
    

    def forward(self, input_ids, attention_masks):
        out = self.bert(input_ids, attention_masks).last_hidden_state
        out = self.fc(out)
        return out

    
    def _shared_evaluation_step(self, batch, batch_idx):
        ids, masks, lbls = batch
        emissions = self(ids, masks)
        loss = -self.crf(emissions, lbls, mask=masks)
        pred = self.crf.decode(emissions, mask=masks)
        r, p, f1 = f1score(lbls, pred, model="transformer")
        return loss, r, p, f1
    

    def test_step(self, batch, batch_idx):
        loss, r, p, f1 = self._shared_evaluation_step(batch, batch_idx)
        self.log("test_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_recall", r, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_precision", p, on_step=False, on_epoch=True, prog_bar=True)
        self.log("test_f1score", f1, on_step=False, on_epoch=True, prog_bar=True)

### Use TF-IDF_SVM Model for predicting Suggestions

In [ ]:
with open("../../../src_sug/saved_weights/tfidf_vectorizer_ner.pkl", 'rb') as f:
    vectorizer = load(f)

with open("../../../src_sug/saved_weights/svc_ner.pkl", 'rb') as f:
    svc_classifier = load(f)

In [ ]:
df_ner_train = pd.read_csv("../../../data/train_290818.txt", 
                           sep=' ',
                           header=None,
                           names=['a', 'b', 'c'],
                           encoding="utf-8",
                           converters={'a': pd.eval, 
                                       'b': pd.eval})

df_ner_train['c'] = df_ner_train['c'].apply(lambda x: 0 if not x else 1)
df_ner_train['a'] = df_ner_train['a'].apply(lambda x: ' '.join(x))

Xtrain_ner = vectorizer.fit_transform([x for x in df_ner_train['a']])
ytrain_ner = df_ner_train['c'].to_numpy()

In [ ]:
svc_classifier.fit(Xtrain_ner, ytrain_ner)

In [ ]:
df_ner_test = pd.read_csv("../../../data/test_290818.txt", 
                          sep=' ',
                          header=None,
                          names=['a', 'b', 'c'],
                          encoding="utf-8",
                          converters={'a': pd.eval, 
                                      'b': pd.eval})

df_ner_test['c'] = df_ner_test['c'].apply(lambda x: 0 if not x else 1)
df_ner_test['a'] = df_ner_test['a'].apply(lambda x: ' '.join(x))

Xtest_ner = vectorizer.transform([x for x in df_ner_test['a']])
ytest_ner = df_ner_test['c'].to_numpy()

In [ ]:
ypred_ner = svc_classifier.predict(Xtest_ner)
print(f"test f1_score (NER): {f1_score(ytest_ner, ypred_ner, zero_division=0):.4f}\n")
print(classification_report(ytest_ner, ypred_ner, target_names=['Positive', 'Negative'], digits=4))

### Use the sentences which are predicted to have a Suggestion as input to NER Model

In [ ]:
sug_idx = torch.LongTensor([i for (i, y) in enumerate(ypred_ner) if y == 1])

In [ ]:
# similarly read the test data and create the dataset
encoded_input, extended_labels = get_encoded_input("../../../data/test_290818.txt", 
                                                   tag2idx=TAG2IDX, 
                                                   tokenizer_name=BERT_TYPE)

test_dataset = TensorDataset(torch.index_select(torch.LongTensor(encoded_input["input_ids"]), dim=0, index=sug_idx),
                             torch.index_select(torch.BoolTensor(encoded_input["attention_mask"]), dim=0, index=sug_idx),
                             torch.index_select(torch.LongTensor(extended_labels), dim=0, index=sug_idx))

In [ ]:
model = BERT_NER(bert_type=BERT_TYPE,
                 test_dataset=test_dataset)

trainer = pl.Trainer(accelerator="gpu",
                     precision=16,
                     log_every_n_steps=1)

In [ ]:
model.load_state_dict(torch.load(f"../../saved_weights/{BERT_TYPE}-ner.ckpt")["state_dict"])
trainer.test(model)